In [112]:
%matplotlib inline
import cv2
import numpy as np
import sys
import glob
import math
import os
from matplotlib import pyplot as plt
from skimage.feature import hog
from skimage import data, color, exposure
import pickle
from operator import itemgetter

In [113]:
 modle = pickle.load(open('train_numberplate.xml', "rb"))

In [3]:
def getHOGDescriptor(image):
    #im = cv2.imread(imagePath)
    gr = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    gr1 = cv2.resize(gr,(64*2,64*2))

    fd, hog_image = hog(gr1, orientations=9, pixels_per_cell=(4, 4),
                    cells_per_block=(1, 1),visualise=True)
    return fd

In [135]:
def displayDetectedChar(path,licencePltImage,image,x1,y1):
    blobFilePath = path + '/blobdetectionFile.txt'
    c = 0
    l = []
    with open(blobFilePath,'r') as file:
        for line in file:
            rect =  line.split()
            x = x1 + int(rect[0])
            y = y1 + int(rect[1])
            width =  int(rect[2])
            height = int(rect[3])
            #cv2.rectangle(image,(x,y), (x+width,y+height),(255,0,0), 1)
            blob = image[y:y+height,x:x+width]
            fd = getHOGDescriptor(blob)
            fd = np.array(fd)
            fd = fd.reshape(-1,fd.size)
            #print np.array(fd).shape
            output_pred = modle.predict(fd)
            l.append([x,y,str(output_pred[0])])
            #print output_pred
            #print type(output_pred)
            c += 20
            
            #cv2.putText(image,output_pred[0],(x + c,y-20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            
    print l
    l.sort(key = lambda x: x[0])
    for draw in l:
        x1 += 20
        cv2.putText(image,draw[2],(x1,y1),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
        
    

In [136]:
def predictEachChar(image,x1,x2,y1,y2):
    os.system('rm -rf /home/spratik/OCR_HOG/Executable/temp')
    os.system('mkdir /home/spratik/OCR_HOG/Executable/temp')
    tempPath = '/home/spratik/OCR_HOG/Executable/temp'
    croppedImagePath = tempPath + '/licencePlt.png'
    licencePlt = image[y1:y2,x1:x2]
    cv2.imwrite(croppedImagePath,licencePlt)
    path =  '/home/spratik/OCR_HOG/Executable/BlobDetection '+ croppedImagePath + ' ' + tempPath
    #print path
    os.system(path)
    displayDetectedChar(tempPath,licencePlt, image,x1,y1)
    
    
    
    

In [137]:
def processImageHaar(image,licenceplt):
    
    for (x,y,w,h) in licenceplt:
        print x,y,w,h
        #cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
        predictEachChar(image, x,x + w,y,y + h)
    
    
    cv2.imwrite('/home/spratik/OCR_HOG/output/output.png',image)     
    

In [138]:
image = cv2.imread('/home/spratik/OCR_HOG/output_image/IMAGE_18.png')
licence = cv2.CascadeClassifier('/home/spratik/OCR_HOG/licenseplate.xml')
gray = cv2.cvtColor(image,0)
licenceplt  = licence.detectMultiScale(gray,1.3,5)
print licenceplt
processImageHaar(gray,licenceplt)


[[548 245 149  56]]
548 245 149 56
[[574, 265, 'K'], [599, 265, '5'], [624, 265, 'N'], [639, 265, '2'], [660, 265, '1'], [669, 265, 'S'], [586, 266, 'A'], [611, 266, '3'], [650, 266, '1']]


In [16]:
# Create a VideoCapture object
cap = cv2.VideoCapture('/home/spratik/Computer-Vision/INPUT_VIDEO/VID20180707152732.mp4')
 
# Check if camera opened successfully
if (cap.isOpened() == False): 
  print("Unable to read camera feed")
 
# Default resolutions of the frame are obtained.The default resolutions are system dependent.
# We convert the resolutions from float to integer.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
 
# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
#out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
c = 0
while(True):
  ret, frame = cap.read()
 
  if ret == True: 
    #outputImage = process_image(frame, 0, type='rect')
    # Write the frame into the file 'output.avi'
    #out.write(outputImage)
    # Display the resulting frame    
    #cv2.imshow('frame',frame)
    name = '/home/spratik/OCR_HOG/output_image/IMAGE_'+ str(c) + '.png'
    c +=1
    cv2.imwrite(name,frame)
 
    # Press Q on keyboard to stop recording
    #if cv2.waitKey(1) & 0xFF == ord('q'):
     # break
 
  # Break the loop
  else:
    break 
 
# When everything done, release the video capture and video write objects
cap.release()
#out.release()
 
# Closes all the frames
#cv2.destroyAllWindows() 

NameError: name 'out' is not defined

In [36]:
# Create a VideoCapture object
cap = cv2.VideoCapture('/home/spratik/Computer-Vision/INPUT_VIDEO/VID20180707152732.mp4')
 
# Check if camera opened successfully
if (cap.isOpened() == False): 
  print("Unable to read camera feed")
 
# Default resolutions of the frame are obtained.The default resolutions are system dependent.
# We convert the resolutions from float to integer.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
 
# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
licence = cv2.CascadeClassifier('/home/spratik/OCR_HOG/licenseplate.xml')
while(True):
  ret, frame = cap.read()
 
  if ret == True: 
    
    outputImage = process_image(frame, 0, type='rect')
    # Write the frame into the file 'output.avi'
    out.write(outputImage)
 
    # Display the resulting frame    
    #cv2.imshow('frame',frame)
 
    # Press Q on keyboard to stop recording
    #if cv2.waitKey(1) & 0xFF == ord('q'):
     # break
 
  # Break the loop
  else:
    break 
 
# When everything done, release the video capture and video write objects
cap.release()
out.release()
 
# Closes all the frames
cv2.destroyAllWindows() 